In [ ]:
import numpy as np
np.set_printoptions(precision=3)
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:.3f}') 
import datetime

import matplotlib as mpl
mpl.rcParams['figure.figsize']=(20,8)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# 기존 코드
mpl.rcParams['axes.grid'] = True

# 새로운 코드
# ax = plt.gca()  # 현재 축을 가져옵니다.
# ax.xaxis.set_major_locator(ticker.MultipleLocator(24))  # x축의 주요 눈금을 24마다 설정합니다.
import matplotlib.pyplot as plt
import seaborn as sns
def setting_styles_basic():
    from matplotlib import rcParams # 한글 환경 설정을 위한 rcParams 임포트
    rcParams['font.family'] = 'Malgun Gothic'
    rcParams['axes.unicode_minus'] = False # 한글 폰트 사용 시, 마이너스 기호가 깨지는 현상 방지

setting_styles_basic()

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor


import statsmodels
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa.stattools import grangercausalitytests
# from statsmodels.tsa.vector_ar.var_model import VAR,LagOrderResults
from statsmodels.tsa.vector_ar.vecm import VECM,select_coint_rank, select_order,coint_johansen
# from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa import seasonal

import optuna

In [ ]:
df=pd.read_csv('./merged_dataset.csv', index_col=0)
# df['datetime']=pd.to_datetime(df['datetime'])
print(df.shape)
print(df.columns)

In [ ]:
df['datetime']=pd.to_datetime(df['datetime'])
data=df.groupby(['data_block_id','datetime','is_consumption']).mean()[['target','direct_solar_radiation_f','surface_solar_radiation_downwards_f',
                                                              'cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f','cloudcover_total_f',
                                                              'dewpoint_f','temperature_f']]
data=data.reset_index().set_index('datetime')
data_prod=data[data['is_consumption']==0]
for col in data_prod.drop(['data_block_id','is_consumption'], axis=1).columns:
    sns.lineplot(data_prod,x=data_prod.index, y=col, color='sandybrown')
    plt.title(f'{col}_original')
    plt.show()
    sns.lineplot(data_prod.diff(24).dropna(),x=data_prod.index[24:], y=col, color='forestgreen')
    plt.title(f'{col}_diff24')
    plt.show()
    data_prod_sc=pd.DataFrame(StandardScaler().fit_transform(data_prod.diff(24).dropna()), columns=data_prod.diff(24).dropna().columns,
                             index=data_prod.diff(24).dropna().index)
    sns.lineplot(data_prod_sc, x=data_prod_sc.index, y=col, color='coral')
    plt.title(f'{col}_diff24_standard')
    plt.show()
    # sns.lineplot(data_prod.diff(24).dropna(),x=data_prod.index[24:], y=col, color='blueviolet')
    # plt.title(f'{col}_diff24')
    # plt.show()

# VECMX

## 함수 정리

In [ ]:


def process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column, diff=24): # 
    print(f'소비여부: {bool(is_consumption)}, 프로슈머유형: {prediction_unit_id}, 테스트날짜: {test_data_block_id}')
    # datetime을 시간 주기 datetime 객체로 변환
    df['datetime']=pd.to_datetime(df['datetime'])
    df['datetime_H'] = pd.DatetimeIndex(df['datetime']).to_period('H')
    # 조건에 맞는 데이터프레임만 가져오기 + 시간 인덱스 설정
    df=df[((df['is_consumption']==is_consumption)&(df['prediction_unit_id']==prediction_unit_id))].set_index('datetime_H')
    # 타겟 및 시간 컬럼을 제외한 사용 컬럼에 대한 24차분을 통해 하루전날 대비 변화량으로 표현
    if diff:
        data=df[variables_columns].diff(diff) # .drop(target_column, axis=1)
        
    # data_block_id를 따오기
    data['data_block_id']=df['data_block_id']
    # # 타겟 컬럼추가
    # data[target_column]=df[target_column]
    data=data.dropna()

    # 지구 자전 컬럼 추가
    data['earth_rotation_degree']=data.index.hour.values
    data['earth_rotation_degree']=data['earth_rotation_degree'].apply(lambda x: x if x<=12 else 24-x)

    # 지구의 태양 공전 궤도를 0~1 사이로 스케일링
    def scale_date(year, dayofyear):
        def solstice_date(year):
            return 90 + 0.2422 * (year - 2000) - (year - 2000) // 4
        
        # 하지와 동지의 태양력 날짜를 구하기
        summer_solstice = solstice_date(year) 
        winter_solstice = solstice_date(year) + 181
    
        # 태양력으로 된 날짜를 하지를 최대값으로, 동지를 최소값으로 하여 스케일링하기
        scaled_date = (dayofyear - winter_solstice) / (summer_solstice - winter_solstice)
    
        # 스케일링된 날짜를 반환하기
        return scaled_date    
    # 지구의 태양 공전 컬럼 추가
    data['earth_orbit_degree']=scale_date(data.index.year, data.index.dayofyear)
    

    
    # train=data[data['data_block_id']<=369].drop('data_block_id', axis=1) ### 1년만 학습 -> 결과 구데기임. 
    train=data.drop(data[data['data_block_id']>=test_data_block_id].index).drop('data_block_id', axis=1) ### data_block_id를 바꿈. 테스트 이후의 날짜는 사용하면 안돼
    test=data[data['data_block_id']==test_data_block_id].drop('data_block_id', axis=1) ############################ original(하루 예측용)
    # test=data[data['data_block_id']>=test_data_block_id].drop('data_block_id', axis=1) ############################ modify(634이후 4일 예측용)
    

    print('columns: ', train.columns)
    return train, test

def scale_time_df(train, test, target_column):
    x_scaler=StandardScaler()
    train_sc=pd.DataFrame(x_scaler.fit_transform(train.drop(target_column, axis=1)), index=train.index, 
                                                   columns= train.drop(target_column, axis=1).columns)
    test_sc=pd.DataFrame(x_scaler.transform(test.drop(target_column, axis=1)), index=test.index,
                          columns=test.drop(target_column, axis=1).columns)

    def relu(x):
        return np.max(x,0)
    
    train_sc['earth_rotation_degree']=train_sc['earth_rotation_degree'].apply(lambda x: relu(x)) ##### relu 통해서 야간 시단개의 영향력을 제거
    test_sc['earth_rotation_degree']=test_sc['earth_rotation_degree'].apply(lambda x: relu(x)) ##### relu 통해서 야간 시단개의 영향력을 제거

    y_scaler=StandardScaler()
    y_train_sc=pd.Series(y_scaler.fit_transform(train[target_column].values.reshape(-1,1))[:,0], index=train.index)
    y_test_sc=pd.Series(y_scaler.transform(test[target_column].values.reshape(-1,1))[:,0], index=test.index)

    
    train_sc['target']=y_train_sc
    test_sc['target']=y_test_sc

    return train_sc, test_sc, y_scaler

def vecmx_modeling(train_df, endog_columns, k_ar_diff, exog_columns=None, exog_coint_columns=None, 
                   n_order=False, maxlags=None, deterministic='n'):
    if n_order:
        lag_order = select_order(train_df[endog_columns], maxlags=maxlags, deterministic=deterministic)
        print('best aic lag order: ', lag_order.aic)
        k_ar_diff=lag_order.aic

    print('k_ar_diff: ', k_ar_diff)
    coint_result=statsmodels.tsa.vector_ar.vecm.select_coint_rank(train_df[endog_columns], k_ar_diff=k_ar_diff, det_order=-1)
    coint_rank=coint_result.rank
    print('coint_rank: ', coint_rank)
    print(coint_result)

    if exog_columns and exog_coint_columns: 
        model=VECM(endog=train_df[endog_columns], exog=train_df[exog_columns], exog_coint=train_df[exog_coint_columns],
                   k_ar_diff=k_ar_diff, coint_rank=coint_rank, deterministic=deterministic)
    elif exog_columns and not exog_coint_columns:
        model=VECM(endog=train_df[endog_columns], exog=train_df[exog_columns],
               k_ar_diff=k_ar_diff, coint_rank=coint_rank, deterministic=deterministic)
    elif not exog_columns and exog_coint_columns:
        model=VECM(endog=train_df[endog_columns], exog_coint=train_df[exog_coint_columns],
                   k_ar_diff=k_ar_diff, coint_rank=coint_rank, deterministic=deterministic)
    else:
        model=VECM(endog=train_df[endog_columns],
                   k_ar_diff=k_ar_diff, coint_rank=coint_rank, deterministic=deterministic)
        
    model_fit=model.fit()
    
    if not(exog_columns or exog_coint_columns):
        # ax = plt.gca()  # 현재 축을 가져옵니다.
        # ax.xaxis.set_major_locator(ticker.MultipleLocator(24))
        model_fit.plot_forecast(steps=24, n_last_obs=96,plot_conf_int=False)
        plt.show()
        
    return model_fit

# 만들어진 모델로 예측해줌. 예측값을 다시 원래의 스케일로 되돌려서 반환.
def vecmx_predicting(model_fit, y_scaler,test_sc=None,exog_columns=None, exog_coint_columns=None, steps=24, nth_day=0):
    if exog_columns and exog_coint_columns: 
        y_pred=model_fit.predict(steps=steps, exog_coint_fc=test_sc[exog_coint_columns].iloc[nth_day*steps:nth_day*steps+steps], 
                             exog_fc=test_sc[exog_columns].iloc[nth_day*steps:nth_day*steps+steps])
    elif exog_columns and not exog_coint_columns:
        y_pred=model_fit.predict(steps=steps, 
                             exog_fc=test_sc[exog_columns].iloc[nth_day*steps:nth_day*steps+steps])
    elif not exog_columns and exog_coint_columns:
        y_pred=model_fit.predict(steps=steps, exog_coint_sc=test_df[exog_coint_columns].iloc[nth_day*steps:nth_day*steps+steps])
    else:
        y_pred=model_fit.predict(steps=steps)

    
    return y_pred

# r2, mae, rmse 평가지표 뽑아주고, test용 data_block_id에 해당하는 날에 대한 그래프 그려주고, 시간을 인덱스로 실제값, 시간, 예측값을 담은 데이터 프레임 반환해줌.
def vecmx_evaluating(test,endog_columns, y_pred, prediction_unit_id, steps=24, nth_day=0, target_location=0, savefig=False): 
    
    endog_test=test[endog_columns]
    test_df=endog_test.iloc[nth_day*steps:nth_day*steps+steps]
    test_df['prediction']=y_pred[:,target_location]

    r2=r2_score(test_df['target'],test_df['prediction'])
    mae=mean_absolute_error(test_df['target'],test_df['prediction'])
    rmse= np.sqrt(mean_squared_error(test_df['target'],test_df['prediction']))
    
    print('r2_score: ', r2)
    print('mae: ', mae)
    print('rmse: ', rmse)
    
    dt=test_df.index[0]
    year = dt.year
    month = dt.month
    day = dt.day
    dt=f'{year}년_{month}월_{day}일'

    plt.figure(figsize=(20,8))
    sns.lineplot(test_df, x=test_df.index.to_timestamp(), y='target', label='actual', color='tomato') # 최애 색: tamato, deepskyblue
    sns.lineplot(test_df, x=test_df.index.to_timestamp(), y='prediction', label='prediction', color='deepskyblue')
    # plt.title(f'프로슈머유형{prediction_unit_id}번_{dt}_태양광발전량예측\nR2: {r2:.3f}, MAE: {mae:.3f}, RMSE: {rmse:.3f}')
    plt.title(f'프로슈머유형{prediction_unit_id}번_{dt}_태양광발전량예측')
    if savefig: # 만약 그래프를 파일로 저장하고 싶다면, 경로 잘 지정하고, savefig 매개변수 True로 설정해주기.
        plt.savefig(f'./VECM/프로슈머유형{prediction_unit_id}번_{dt}_24시간예측량.jpg')
    plt.show()


    return test_df

In [ ]:
train_sc['earth_rotation_degree'].apply(lambda x: x if x>0 else 0)

In [ ]:
def relu(x):
    return np.max(x,0)
is_consumption=0 
test_data_block_id=574
prediction_unit_id=5
variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
             'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
target_column='target'

endog_columns=['target','surface_solar_radiation_downwards_f','earth_rotation_degree']
k_ar_diff=360

train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)
train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)
train_sc['earth_rotation_degree']=train_sc['earth_rotation_degree'].apply(lambda x: x if x>0 else 0)
test_sc['earth_rotation_degree']=test_sc['earth_rotation_degree'].apply(lambda x: x if x>0 else 0)
model_fit=vecmx_modeling(train_sc, endog_columns, k_ar_diff, exog_columns=None, exog_coint_columns=None, 
                   n_order=False, maxlags=None, deterministic='n')
y_pred=vecmx_predicting(model_fit,y_scaler,test_sc=None, exog_columns=None, exog_coint_columns=None, steps=24, nth_day=0)
test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=24, nth_day=0, target_location=0, savefig=False)
# model_fit.plot_forecast(steps=24, n_last_obs=72,plot_conf_int=False)

In [ ]:
# deterministic=coli
is_consumption=0
test_data_block_id=496
prediction_unit_id=5
variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
             'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
target_column='target'

k_ar_diff=360
endog_columns=['target','earth_rotation_degree']
exog_coint_columns=['direct_solar_radiation_f','cloudcover_low_f','snowfall_f']
exog_columns=['earth_orbit_degree','surface_solar_radiation_downwards_f','cloudcover_total_f','dewpoint_f']

train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)

train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)

model_fit=vecmx_modeling(train_sc, endog_columns=endog_columns, k_ar_diff=k_ar_diff, exog_coint_columns=exog_coint_columns, exog_columns=exog_columns, 
                   n_order=False, maxlags=None, deterministic='coli')

y_pred=vecmx_predicting(model_fit,y_scaler,test_sc, steps=24, exog_coint_columns=exog_coint_columns, exog_columns=exog_columns)

test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=24, nth_day=0, target_location=0, savefig=False)

In [ ]:
# dbi 635
is_consumption=0
test_data_block_id=635
prediction_unit_id=5
variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
             'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
target_column='target'

k_ar_diff=360
endog_columns=['target','earth_rotation_degree']
exog_coint_columns=['direct_solar_radiation_f','cloudcover_low_f','snowfall_f']
exog_columns=['earth_orbit_degree','surface_solar_radiation_downwards_f','cloudcover_total_f','dewpoint_f']

train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)

train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)

model_fit=vecmx_modeling(train_sc, endog_columns=endog_columns, k_ar_diff=k_ar_diff, exog_coint_columns=exog_coint_columns, exog_columns=exog_columns, 
                   n_order=False, maxlags=None, deterministic='coli')

y_pred=vecmx_predicting(model_fit,y_scaler,test_sc, steps=24, exog_coint_columns=exog_coint_columns, exog_columns=exog_columns)

test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=24, nth_day=0, target_location=0, savefig=False)

## dbi 634~637 예측

In [ ]:
# exog 고려 x
is_consumption=0 
test_data_block_id=634
prediction_unit_id=5
variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
             'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
target_column='target'

endog_columns=['target','earth_rotation_degree']
k_ar_diff=360

train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)
train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)
model_fit=vecmx_modeling(train_sc, endog_columns, k_ar_diff, exog_columns=None, exog_coint_columns=None, 
                   n_order=False, maxlags=None, deterministic='n')
y_pred=vecmx_predicting(model_fit,y_scaler,test_sc=None, exog_columns=None, exog_coint_columns=None, steps=len(test), nth_day=0)
test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=len(test), nth_day=0, target_location=0, savefig=False)
# model_fit.plot_forecast(steps=24, n_last_obs=72,plot_conf_int=False)

In [ ]:
# 데이터 블럭 아이디 바꾸고, exog 다 포함해서
is_consumption=0
test_data_block_id=634
prediction_unit_id=5
variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
             'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
target_column='target'

k_ar_diff=360
endog_columns=['target','earth_rotation_degree']
exog_coint_columns=['direct_solar_radiation_f','cloudcover_low_f','snowfall_f']
exog_columns=['earth_orbit_degree','surface_solar_radiation_downwards_f','cloudcover_total_f','dewpoint_f']

train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)

train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)

model_fit=vecmx_modeling(train_sc, endog_columns=endog_columns, k_ar_diff=k_ar_diff, exog_coint_columns=exog_coint_columns, exog_columns=exog_columns, 
                   n_order=False, maxlags=None, deterministic='n')

y_pred=vecmx_predicting(model_fit,y_scaler,test_sc, steps=len(test), exog_coint_columns=exog_coint_columns, exog_columns=exog_columns)

test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=len(test), nth_day=0, target_location=0, savefig=False)

In [ ]:
test_df['target_original']=y_scaler.inverse_transform(test_df[['target','prediction']])[:,0]
test_df['prediction_original']=y_scaler.inverse_transform(test_df[['target','prediction']])[:,1]

## 테스트: 랜덤 샘플링 검증

In [ ]:

for p in np.random.randint(0,60,3):
    for d in np.random.randint(400,638,5):
        try:
            is_consumption=0
            test_data_block_id=d
            prediction_unit_id=p
            variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
                         'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
            target_column='target'
            
            k_ar_diff=360
            endog_columns=['target','earth_rotation_degree']
            exog_coint_columns=['direct_solar_radiation_f','cloudcover_low_f','snowfall_f']
            exog_columns=['earth_orbit_degree','surface_solar_radiation_downwards_f','cloudcover_total_f','dewpoint_f']
            
            train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)
            
            train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)
            
            model_fit=vecmx_modeling(train_sc, endog_columns=endog_columns, k_ar_diff=k_ar_diff, exog_coint_columns=exog_coint_columns, exog_columns=exog_columns, 
                               n_order=False, maxlags=None, deterministic='n')
            
            y_pred=vecmx_predicting(model_fit,y_scaler,test_sc, steps=len(test), exog_coint_columns=exog_coint_columns, exog_columns=exog_columns)
            
            test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=len(test), nth_day=0, target_location=0, savefig=False)
        except:
            continue

In [ ]:

for p in np.random.randint(0,60,3):
    for d in np.random.randint(400,638,5):
        try:
            is_consumption=0
            test_data_block_id=d
            prediction_unit_id=p
            variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
                         'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
            target_column='target'
            
            k_ar_diff=360
            endog_columns=['target','earth_rotation_degree']
            exog_coint_columns=['direct_solar_radiation_f','cloudcover_low_f','snowfall_f']
            exog_columns=['earth_orbit_degree','surface_solar_radiation_downwards_f','cloudcover_total_f','dewpoint_f']
            
            train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)
            
            train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)
            
            model_fit=vecmx_modeling(train_sc, endog_columns=endog_columns, k_ar_diff=k_ar_diff, exog_coint_columns=exog_coint_columns, exog_columns=exog_columns, 
                               n_order=False, maxlags=None, deterministic='n')
            
            y_pred=vecmx_predicting(model_fit,y_scaler,test_sc, steps=len(test), exog_coint_columns=exog_coint_columns, exog_columns=exog_columns)
            
            test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=len(test), nth_day=0, target_location=0, savefig=False)
        except:
            continue

In [ ]:

for p in np.random.randint(0,60,3):
    for d in np.random.randint(400,638,5):
        try:
            is_consumption=0
            test_data_block_id=d
            prediction_unit_id=p
            variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
                         'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
            target_column='target'
            
            k_ar_diff=360
            endog_columns=['target','earth_rotation_degree']
            exog_coint_columns=['direct_solar_radiation_f','cloudcover_low_f','snowfall_f']
            exog_columns=['earth_orbit_degree','surface_solar_radiation_downwards_f','cloudcover_total_f','dewpoint_f']
            
            train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)
            
            train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)
            
            model_fit=vecmx_modeling(train_sc, endog_columns=endog_columns, k_ar_diff=k_ar_diff, exog_coint_columns=exog_coint_columns, exog_columns=exog_columns, 
                               n_order=False, maxlags=None, deterministic='n')
            
            y_pred=vecmx_predicting(model_fit,y_scaler,test_sc, steps=len(test), exog_coint_columns=exog_coint_columns, exog_columns=exog_columns)
            
            test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=len(test), nth_day=0, target_location=0, savefig=False)
        except:
            continue

In [ ]:
weather_forecast_diff(df)

## 내생변수: 지표면하향복사에너지 추가

In [ ]:
def process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column, diff=24): # 
    print(f'소비여부: {bool(is_consumption)}, 프로슈머유형: {prediction_unit_id}, 테스트날짜: {test_data_block_id}')
    # datetime을 시간 주기 datetime 객체로 변환
    df['datetime']=pd.to_datetime(df['datetime'])
    df['datetime_H'] = pd.DatetimeIndex(df['datetime']).to_period('H')
    # 조건에 맞는 데이터프레임만 가져오기 + 시간 인덱스 설정
    df=df[((df['is_consumption']==is_consumption)&(df['prediction_unit_id']==prediction_unit_id))].set_index('datetime_H')
    # 타겟 및 시간 컬럼을 제외한 사용 컬럼에 대한 24차분을 통해 하루전날 대비 변화량으로 표현
    if diff:
        data=df[variables_columns].diff(diff) # .drop(target_column, axis=1)
        
    # data_block_id를 따오기
    data['data_block_id']=df['data_block_id']
    # # 타겟 컬럼추가
    # data[target_column]=df[target_column]
    data=data.dropna()

    # 지구 자전 컬럼 추가
    data['earth_rotation_degree']=data.index.hour.values
    data['earth_rotation_degree']=data['earth_rotation_degree'].apply(lambda x: x if x<=12 else 24-x)

    # 지구의 태양 공전 궤도를 0~1 사이로 스케일링
    def scale_date(year, dayofyear):
        def solstice_date(year):
            return 90 + 0.2422 * (year - 2000) - (year - 2000) // 4
        
        # 하지와 동지의 태양력 날짜를 구하기
        summer_solstice = solstice_date(year) 
        winter_solstice = solstice_date(year) + 181
    
        # 태양력으로 된 날짜를 하지를 최대값으로, 동지를 최소값으로 하여 스케일링하기
        scaled_date = (dayofyear - winter_solstice) / (summer_solstice - winter_solstice)
    
        # 스케일링된 날짜를 반환하기
        return scaled_date    
    # 지구의 태양 공전 컬럼 추가
    data['earth_orbit_degree']=scale_date(data.index.year, data.index.dayofyear)
    

    
    # train=data[data['data_block_id']<=369].drop('data_block_id', axis=1) ### 1년만 학습 -> 결과 구데기임. 
    train=data.drop(data[data['data_block_id']>=test_data_block_id].index).drop('data_block_id', axis=1) ### data_block_id를 바꿈. 테스트 이후의 날짜는 사용하면 안돼
    test=data[data['data_block_id']==test_data_block_id].drop('data_block_id', axis=1) ############################ original(하루 예측용)
    # test=data[data['data_block_id']>=test_data_block_id].drop('data_block_id', axis=1) ############################ modify(634이후 4일 예측용)
    

    print('columns: ', train.columns)
    return train, test

def scale_time_df(train, test, target_column):
    x_scaler=StandardScaler()
    train_sc=pd.DataFrame(x_scaler.fit_transform(train.drop(target_column, axis=1)), index=train.index, 
                                                   columns= train.drop(target_column, axis=1).columns)
    test_sc=pd.DataFrame(x_scaler.transform(test.drop(target_column, axis=1)), index=test.index,
                          columns=test.drop(target_column, axis=1).columns)

    y_scaler=StandardScaler()
    y_train_sc=pd.Series(y_scaler.fit_transform(train[target_column].values.reshape(-1,1))[:,0], index=train.index)
    y_test_sc=pd.Series(y_scaler.transform(test[target_column].values.reshape(-1,1))[:,0], index=test.index)

    train_sc['target']=y_train_sc
    test_sc['target']=y_test_sc

    return train_sc, test_sc, y_scaler

def vecmx_modeling(train_df, endog_columns, k_ar_diff, exog_columns=None, exog_coint_columns=None, 
                   n_order=False, maxlags=None, deterministic='n'):
    if n_order:
        lag_order = select_order(train_df[endog_columns], maxlags=maxlags, deterministic=deterministic)
        print('best aic lag order: ', lag_order.aic)
        k_ar_diff=lag_order.aic

    print('k_ar_diff: ', k_ar_diff)
    coint_result=statsmodels.tsa.vector_ar.vecm.select_coint_rank(train_df[endog_columns], k_ar_diff=k_ar_diff, det_order=-1)
    coint_rank=coint_result.rank
    print('coint_rank: ', coint_rank)
    print(coint_result)

    if exog_columns and exog_coint_columns: 
        model=VECM(endog=train_df[endog_columns], exog=train_df[exog_columns], exog_coint=train_df[exog_coint_columns],
                   k_ar_diff=k_ar_diff, coint_rank=coint_rank, deterministic=deterministic)
    elif exog_columns and not exog_coint_columns:
        model=VECM(endog=train_df[endog_columns], exog=train_df[exog_columns],
               k_ar_diff=k_ar_diff, coint_rank=coint_rank, deterministic=deterministic)
    elif not exog_columns and exog_coint_columns:
        model=VECM(endog=train_df[endog_columns], exog_coint=train_df[exog_coint_columns],
                   k_ar_diff=k_ar_diff, coint_rank=coint_rank, deterministic=deterministic)
    else:
        model=VECM(endog=train_df[endog_columns],
                   k_ar_diff=k_ar_diff, coint_rank=coint_rank, deterministic=deterministic)
        
    model_fit=model.fit()
    
    if not(exog_columns or exog_coint_columns):
        # ax = plt.gca()  # 현재 축을 가져옵니다.
        # ax.xaxis.set_major_locator(ticker.MultipleLocator(24))
        model_fit.plot_forecast(steps=24, n_last_obs=96,plot_conf_int=False)
        plt.show()
        
    return model_fit

# 만들어진 모델로 예측해줌. 예측값을 다시 원래의 스케일로 되돌려서 반환.
def vecmx_predicting(model_fit, y_scaler,test_sc=None,exog_columns=None, exog_coint_columns=None, steps=24, nth_day=0):
    if exog_columns and exog_coint_columns: 
        y_pred=model_fit.predict(steps=steps, exog_coint_fc=test_sc[exog_coint_columns].iloc[nth_day*steps:nth_day*steps+steps], 
                             exog_fc=test_sc[exog_columns].iloc[nth_day*steps:nth_day*steps+steps])
    elif exog_columns and not exog_coint_columns:
        y_pred=model_fit.predict(steps=steps, 
                             exog_fc=test_sc[exog_columns].iloc[nth_day*steps:nth_day*steps+steps])
    elif not exog_columns and exog_coint_columns:
        y_pred=model_fit.predict(steps=steps, exog_coint_sc=test_df[exog_coint_columns].iloc[nth_day*steps:nth_day*steps+steps])
    else:
        y_pred=model_fit.predict(steps=steps)

    
    return y_pred

# r2, mae, rmse 평가지표 뽑아주고, test용 data_block_id에 해당하는 날에 대한 그래프 그려주고, 시간을 인덱스로 실제값, 시간, 예측값을 담은 데이터 프레임 반환해줌.
def vecmx_evaluating(test,endog_columns, y_pred, prediction_unit_id, steps=24, nth_day=0, target_location=0, savefig=False): 
    
    endog_test=test[endog_columns]
    test_df=endog_test.iloc[nth_day*steps:nth_day*steps+steps]
    test_df['prediction']=y_pred[:,target_location]

    r2=r2_score(test_df['target'],test_df['prediction'])
    mae=mean_absolute_error(test_df['target'],test_df['prediction'])
    rmse= np.sqrt(mean_squared_error(test_df['target'],test_df['prediction']))
    
    print('r2_score: ', r2)
    print('mae: ', mae)
    print('rmse: ', rmse)
    
    dt=test_df.index[0]
    year = dt.year
    month = dt.month
    day = dt.day
    dt=f'{year}년_{month}월_{day}일'

    plt.figure(figsize=(20,8))
    sns.lineplot(test_df, x=test_df.index.to_timestamp(), y='target', label='actual', color='tomato') # 최애 색: tamato, deepskyblue
    sns.lineplot(test_df, x=test_df.index.to_timestamp(), y='prediction', label='prediction', color='deepskyblue')
    # plt.title(f'프로슈머유형{prediction_unit_id}번_{dt}_태양광발전량예측\nR2: {r2:.3f}, MAE: {mae:.3f}, RMSE: {rmse:.3f}')
    plt.title(f'프로슈머유형{prediction_unit_id}번_{dt}_태양광발전량예측')
    if savefig: # 만약 그래프를 파일로 저장하고 싶다면, 경로 잘 지정하고, savefig 매개변수 True로 설정해주기.
        plt.savefig(f'./VECM/프로슈머유형{prediction_unit_id}번_{dt}_24시간예측량.jpg')
    plt.show()


    return test_df

In [ ]:
# 내생변수 지표면하향복사에너지 추가, 랜덤 검증
for p in np.random.randint(0,68,5):
    for d in np.random.randint(400,627,4):
        try:
            is_consumption=0 
            test_data_block_id=d
            prediction_unit_id=p
            variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
                         'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
            target_column='target'
            
            endog_columns=['target','earth_rotation_degree','surface_solar_radiation_downwards_f']
            k_ar_diff=360
            
            train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)
            train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)
            model_fit=vecmx_modeling(train_sc, endog_columns, k_ar_diff, exog_columns=None, exog_coint_columns=None, 
                               n_order=False, maxlags=None, deterministic='n')
            y_pred=vecmx_predicting(model_fit,y_scaler,test_sc=None, exog_columns=None, exog_coint_columns=None, steps=24, nth_day=0)
            test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=24, nth_day=0, target_location=0, savefig=False)
        except: continue
            # model_fit.plot_forecast(steps=24, n_last_obs=72,plot_conf_int=False)

In [ ]:
# 외생까지 고려
for p in np.random.randint(0,60,10):
    # for d in np.random.randint(370,638,3):
    for d in np.random.randint(500,638,5):
        try:
            is_consumption=0
            test_data_block_id=d
            prediction_unit_id=p
            variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
                         'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
            target_column='target'
            
            k_ar_diff=360
            endog_columns=['target','earth_rotation_degree','surface_solar_radiation_downwards_f']
            exog_coint_columns=['direct_solar_radiation_f','cloudcover_total_f']
            exog_columns=['earth_orbit_degree','dewpoint_f']
            
            train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)
            
            train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)
            
            model_fit=vecmx_modeling(train_sc, endog_columns=endog_columns, k_ar_diff=k_ar_diff, exog_coint_columns=exog_coint_columns, exog_columns=exog_columns, 
                               n_order=False, maxlags=None, deterministic='coli')
            
            y_pred=vecmx_predicting(model_fit,y_scaler,test_sc, steps=len(test), exog_coint_columns=exog_coint_columns, exog_columns=exog_columns)
            
            test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=len(test), nth_day=0, target_location=0, savefig=False)
        except:
            continue

In [ ]:
is_consumption=0 
test_data_block_id=603
prediction_unit_id=43
variables_columns=[ 'target', 'direct_solar_radiation_f', 'surface_solar_radiation_downwards_f', 'dewpoint_f','temperature_f',
             'cloudcover_total_f','cloudcover_low_f','cloudcover_mid_f','cloudcover_high_f', 'snowfall_f', 'rain_f','euros_per_mwh_electricity']
target_column='target'

endog_columns=['target','surface_solar_radiation_downwards_f','earth_rotation_degree']
k_ar_diff=360

train, test= process_time_df(df, is_consumption, test_data_block_id, prediction_unit_id, variables_columns, target_column,  diff=24)
train_sc, test_sc, y_scaler=scale_time_df(train, test, target_column)
model_fit=vecmx_modeling(train_sc, endog_columns, k_ar_diff, exog_columns=None, exog_coint_columns=None, 
                   n_order=False, maxlags=None, deterministic='n')
y_pred=vecmx_predicting(model_fit,y_scaler,test_sc=None, exog_columns=None, exog_coint_columns=None, steps=24, nth_day=0)
test_df=vecmx_evaluating(test_sc,endog_columns, y_pred, prediction_unit_id, steps=24, nth_day=0, target_location=0, savefig=False)
# model_fit.plot_forecast(steps=24, n_last_obs=72,plot_conf_int=False)

# LGBM

In [ ]:
df.set_index('datetime', inplace=True)

In [ ]:
data=df.drop(['county', 'is_business', 'product_type','dt_year', 'dt_month','dt_hour', 'dt_dayofyear'],axis=1)

In [ ]:
data.columns

## 교차검증

In [ ]:
## 소비 모델
kfold=KFold(n_splits=7, shuffle=True)
r2_list=[]
mae_list=[]
rmse_list=[]

data_cons=data[data['is_consumption']==1].drop(['data_block_id','row_id'], axis=1)
cnt=0
for tr_idx, ts_idx in kfold.split(data_cons):
    cnt+=1
    x_train, x_test= data_cons.iloc[tr_idx,1:], data_cons.iloc[ts_idx,1:]
    y_train, y_test= data_cons.iloc[tr_idx,0], data_cons.iloc[ts_idx,0]
    
    x_scaler=MinMaxScaler()
    x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
    x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
    y_scaler=MinMaxScaler()
    y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
    
    # print(tr_idx,ts_idx)
    # display( x_test)
    model=LGBMRegressor(learning_rate= 0.19989546052444535, n_estimators= 300, max_depth= None, 
                        reg_alpha= 0.05673092034527474, metric= 'mae')
    model.fit(x_train_sc, y_train_sc)
    
    feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_name_).sort_values(ascending=False)
    # display(feat_imp)
    # plt.figure(figsize=(24,8))
    sns.barplot(x= feat_imp, y=feat_imp.index)
    plt.title(f'소비모형 교차검증 {cnt}회차 피쳐중요도')
    plt.savefig(f'./최종모델링/lgbm_consmodel_cv{cnt}_feature_importances.jpg')
    plt.show()

    y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
    x_test['Actual']=y_test
    x_test['Prediction']=y_pred
    
    r2=r2_score(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))

    r2_list.append(r2)
    mae_list.append(mae)
    rmse_list.append(rmse)


    print(f'{cnt}회차 r2: {r2}')
    print(f'{cnt}회차 mae: {mae}')
    print(f'{cnt}회차 rmse: {rmse}')

    sns.scatterplot(x_test, x='Actual', y='Prediction', color='tomato')
    sns.lineplot(x_test, x='Actual', y='Actual', color='olive')
    plt.title(f'소비모형 교차검증 {cnt}회차 소비그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
    plt.savefig(f'./최종모델링/lgbm_consmodel_cv{cnt}_consumption.jpg')
    plt.show()
    
    print('-'*100)
    
print(f'교차검증 평균 r2: {np.mean(r2_list)}')
print(f'교차검증 평균 mae: {np.mean(mae_list)}')
print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')

cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
cv_scores['MAE']=mae_list
cv_scores['RMSE']=rmse_list
mean_values = cv_scores.mean()
cv_scores.loc['Average'] = mean_values
cv_scores.to_csv('./최종모델링/lgbm_consmodel_scores.csv')

In [ ]:
## 생산 모델
kfold=KFold(n_splits=7, shuffle=True)
r2_list=[]
mae_list=[]
rmse_list=[]

data_prod=data[data['is_consumption']==0].drop(['data_block_id','row_id'], axis=1)
cnt=0
for tr_idx, ts_idx in kfold.split(data_prod):
    cnt+=1
    x_train, x_test= data_prod.iloc[tr_idx,1:], data_prod.iloc[ts_idx,1:]
    y_train, y_test= data_prod.iloc[tr_idx,0], data_prod.iloc[ts_idx,0]
    
    x_scaler=MinMaxScaler()
    x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
    x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
    y_scaler=MinMaxScaler()
    y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
    
    # print(tr_idx,ts_idx)
    # display( x_test)
    model=LGBMRegressor(learning_rate= 0.19989546052444535, n_estimators= 300, max_depth= None, 
                        reg_alpha= 0.05673092034527474, metric= 'mae')
    model.fit(x_train_sc, y_train_sc)
    
    feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_name_).sort_values(ascending=False)
    # display(feat_imp)
    # plt.figure(figsize=(24,8))
    sns.barplot(x= feat_imp, y=feat_imp.index)
    plt.title(f'생산모형 교차검증 {cnt}회차 피쳐중요도')
    plt.savefig(f'./최종모델링/lgbm_prodmodel_cv{cnt}_feature_importances.jpg')
    plt.show()

    y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
    x_test['Actual']=y_test
    x_test['Prediction']=y_pred
    
    r2=r2_score(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))

    r2_list.append(r2)
    mae_list.append(mae)
    rmse_list.append(rmse)


    print(f'{cnt}회차 r2: {r2}')
    print(f'{cnt}회차 mae: {mae}')
    print(f'{cnt}회차 rmse: {rmse}')

    sns.scatterplot(x_test, x='Actual', y='Prediction', color='deepskyblue')
    sns.lineplot(x_test, x='Actual', y='Actual', color='olive')
    plt.title(f'생산모형 교차검증 {cnt}회차 생산그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
    plt.savefig(f'./최종모델링/lgbm_prodmodel_cv{cnt}_production.jpg')
    plt.show()
    
    print('-'*100)
    
print(f'교차검증 평균 r2: {np.mean(r2_list)}')
print(f'교차검증 평균 mae: {np.mean(mae_list)}')
print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')

cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
cv_scores['MAE']=mae_list
cv_scores['RMSE']=rmse_list
mean_values = cv_scores.mean()
cv_scores.loc['Average'] = mean_values
cv_scores.to_csv('./최종모델링/lgbm_prodmodel_scores.csv')

In [ ]:
## 유형별 소비 모델
for p in [0,2,4,5,31,32,46,48]:
    kfold=KFold(n_splits=7, shuffle=True)
    r2_list=[]
    mae_list=[]
    rmse_list=[]

    data_cons=data[data['is_consumption']==1]
    data_consp=data_cons[data_cons['prediction_unit_id']==p].drop(['is_consumption','data_block_id', 'row_id'],axis=1)
    cnt=0
    for tr_idx, ts_idx in kfold.split(data_consp):
        cnt+=1
        x_train, x_test= data_consp.iloc[tr_idx,1:], data_consp.iloc[ts_idx,1:]
        y_train, y_test= data_consp.iloc[tr_idx,0], data_consp.iloc[ts_idx,0]
        
        x_scaler=MinMaxScaler()
        x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
        x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
        y_scaler=MinMaxScaler()
        y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
        
        # print(tr_idx,ts_idx)
        # display( x_test)
        model=LGBMRegressor(learning_rate= 0.19989546052444535, n_estimators= 300, max_depth= None, 
                            reg_alpha= 0.05673092034527474, metric= 'mae')
        model.fit(x_train_sc, y_train_sc)
        
        feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_name_).sort_values(ascending=False)
        # display(feat_imp)
        # plt.figure(figsize=(24,8))
        sns.barplot(x= feat_imp, y=feat_imp.index)
        plt.title(f'{p}번유형 소비모형 교차검증 {cnt}회차 피쳐중요도')
        plt.savefig(f'./최종모델링/lgbm_type{p}_consmodel_cv{cnt}_feature_importances.jpg')
        plt.show()
    
        y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
        x_test['Actual']=y_test
        x_test['Prediction']=y_pred
        
        r2=r2_score(y_test,y_pred)
        mae=mean_absolute_error(y_test,y_pred)
        rmse=np.sqrt(mean_squared_error(y_test,y_pred))
    
        r2_list.append(r2)
        mae_list.append(mae)
        rmse_list.append(rmse)
    
    
        print(f'{cnt}회차 r2: {r2}')
        print(f'{cnt}회차 mae: {mae}')
        print(f'{cnt}회차 rmse: {rmse}')

        sns.scatterplot(x_test, x='Actual', y='Prediction', color='tomato')
        sns.lineplot(x_test, x='Actual', y='Actual',color='olive')

        # plt.ylabel('consumption')
        plt.title(f'{p}번유형 소비모형 교차검증 {cnt}회차 소비그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
        plt.savefig(f'./최종모델링/lgbm_type{p}_consmodel_cv{cnt}_consumption.jpg')
        plt.show()
        
        print('-'*100)
        
    print(f'교차검증 평균 r2: {np.mean(r2_list)}')
    print(f'교차검증 평균 mae: {np.mean(mae_list)}')
    print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')
    
    cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
    cv_scores['MAE']=mae_list
    cv_scores['RMSE']=rmse_list
    mean_values = cv_scores.mean()
    cv_scores.loc['Average'] = mean_values
    cv_scores.to_csv(f'./최종모델링/lgbm_type{p}_consmodel_scores.csv')

In [ ]:
## 유형별 생산 모델
for p in [0,2,4,5,31,32,46,48]:
    kfold=KFold(n_splits=7, shuffle=True)
    r2_list=[]
    mae_list=[]
    rmse_list=[]

    data_cons=data[data['is_consumption']==0]
    data_consp=data_cons[data_cons['prediction_unit_id']==p].drop(['is_consumption','data_block_id', 'row_id'],axis=1)
    cnt=0
    for tr_idx, ts_idx in kfold.split(data_consp):
        cnt+=1
        x_train, x_test= data_consp.iloc[tr_idx,1:], data_consp.iloc[ts_idx,1:]
        y_train, y_test= data_consp.iloc[tr_idx,0], data_consp.iloc[ts_idx,0]
        
        x_scaler=MinMaxScaler()
        x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
        x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
        y_scaler=MinMaxScaler()
        y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
        
        # print(tr_idx,ts_idx)
        # display( x_test)
        model=LGBMRegressor(learning_rate= 0.19989546052444535, n_estimators= 300, max_depth= None, 
                            reg_alpha= 0.05673092034527474, metric= 'mae')
        model.fit(x_train_sc, y_train_sc)
        
        feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_name_).sort_values(ascending=False)
        # display(feat_imp)
        # plt.figure(figsize=(24,8))
        sns.barplot(x= feat_imp, y=feat_imp.index)
        plt.title(f'{p}번유형 생산모형 교차검증 {cnt}회차 피쳐중요도')
        plt.savefig(f'./최종모델링/lgbm_type{p}_prodmodel_cv{cnt}_feature_importances.jpg')
        plt.show()
    
        y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
        x_test['Actual']=y_test
        x_test['Prediction']=y_pred
        
        r2=r2_score(y_test,y_pred)
        mae=mean_absolute_error(y_test,y_pred)
        rmse=np.sqrt(mean_squared_error(y_test,y_pred))
    
        r2_list.append(r2)
        mae_list.append(mae)
        rmse_list.append(rmse)
    
    
        print(f'{cnt}회차 r2: {r2}')
        print(f'{cnt}회차 mae: {mae}')
        print(f'{cnt}회차 rmse: {rmse}')

        sns.scatterplot(x_test, x='Actual', y='Prediction', color='deepskyblue')
        sns.lineplot(x_test, x='Actual', y='Actual',color='olive')

        # plt.ylabel('consumption')
        plt.title(f'{p}번유형 소비모형 교차검증 {cnt}회차 생산그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
        plt.savefig(f'./최종모델링/lgbm_type{p}_prodmodel_cv{cnt}_production.jpg')
        plt.show()
        
        print('-'*100)
        
    print(f'교차검증 평균 r2: {np.mean(r2_list)}')
    print(f'교차검증 평균 mae: {np.mean(mae_list)}')
    print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')
    
    cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
    cv_scores['MAE']=mae_list
    cv_scores['RMSE']=rmse_list
    mean_values = cv_scores.mean()
    cv_scores.loc['Average'] = mean_values
    cv_scores.to_csv(f'./최종모델링/lgbm_type{p}_prodmodel_scores.csv')

# XGBM

## 교차검증

In [ ]:
## 소비 모델
kfold=KFold(n_splits=7, shuffle=True)
r2_list=[]
mae_list=[]
rmse_list=[]

data_cons=data[data['is_consumption']==1].drop(['data_block_id','row_id'], axis=1)
cnt=0
for tr_idx, ts_idx in kfold.split(data_cons):
    cnt+=1
    x_train, x_test= data_cons.iloc[tr_idx,1:], data_cons.iloc[ts_idx,1:]
    y_train, y_test= data_cons.iloc[tr_idx,0], data_cons.iloc[ts_idx,0]
    
    x_scaler=MinMaxScaler()
    x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
    x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
    y_scaler=MinMaxScaler()
    y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
    
    # print(tr_idx,ts_idx)
    # display( x_test)
    model=XGBRegressor()
    model.fit(x_train_sc, y_train_sc)
    
    feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_names_in_).sort_values(ascending=False)
    # display(feat_imp)
    # plt.figure(figsize=(24,8))
    sns.barplot(x= feat_imp, y=feat_imp.index)
    plt.title(f'소비모형 교차검증 {cnt}회차 피쳐중요도')
    plt.savefig(f'./최종모델링/xgb_consmodel_cv{cnt}_feature_importances.jpg')
    plt.show()

    y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
    x_test['Actual']=y_test
    x_test['Prediction']=y_pred
    
    r2=r2_score(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))

    r2_list.append(r2)
    mae_list.append(mae)
    rmse_list.append(rmse)


    print(f'{cnt}회차 r2: {r2}')
    print(f'{cnt}회차 mae: {mae}')
    print(f'{cnt}회차 rmse: {rmse}')

    sns.scatterplot(x_test, x='Actual', y='Prediction', color='tomato')
    sns.lineplot(x_test, x='Actual', y='Actual', color='olive')
    plt.title(f'소비모형 교차검증 {cnt}회차 소비그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
    plt.savefig(f'./최종모델링/xgb_consmodel_cv{cnt}_consumption.jpg')
    plt.show()
    
    print('-'*100)
    
print(f'교차검증 평균 r2: {np.mean(r2_list)}')
print(f'교차검증 평균 mae: {np.mean(mae_list)}')
print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')

cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
cv_scores['MAE']=mae_list
cv_scores['RMSE']=rmse_list
mean_values = cv_scores.mean()
cv_scores.loc['Average'] = mean_values
cv_scores.to_csv('./최종모델링/xgb_consmodel_scores.csv')

In [ ]:
df.columns

In [ ]:
## 생산 모델
kfold=KFold(n_splits=7, shuffle=True)
r2_list=[]
mae_list=[]
rmse_list=[]

data_prod=data[data['is_consumption']==0].drop(['data_block_id','row_id'], axis=1)
cnt=0
for tr_idx, ts_idx in kfold.split(data_prod):
    cnt+=1
    x_train, x_test= data_prod.iloc[tr_idx,1:], data_prod.iloc[ts_idx,1:]
    y_train, y_test= data_prod.iloc[tr_idx,0], data_prod.iloc[ts_idx,0]
    
    x_scaler=MinMaxScaler()
    x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
    x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
    y_scaler=MinMaxScaler()
    y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
    
    # print(tr_idx,ts_idx)
    # display( x_test)
    model=XGBRegressor()
    model.fit(x_train_sc, y_train_sc)
    
    feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_names_in_).sort_values(ascending=False)
    # display(feat_imp)
    # plt.figure(figsize=(24,8))
    sns.barplot(x= feat_imp, y=feat_imp.index)
    plt.title(f'생산모형 교차검증 {cnt}회차 피쳐중요도')
    plt.savefig(f'./최종모델링/xgb_prodmodel_cv{cnt}_feature_importances.jpg')
    plt.show()

    y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
    x_test['Actual']=y_test
    x_test['Prediction']=y_pred
    
    r2=r2_score(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))

    r2_list.append(r2)
    mae_list.append(mae)
    rmse_list.append(rmse)


    print(f'{cnt}회차 r2: {r2}')
    print(f'{cnt}회차 mae: {mae}')
    print(f'{cnt}회차 rmse: {rmse}')

    sns.scatterplot(x_test, x='Actual', y='Prediction', color='deepskyblue')
    sns.lineplot(x_test, x='Actual', y='Actual', color='olive')
    plt.title(f'생산모형 교차검증 {cnt}회차 생산그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
    plt.savefig(f'./최종모델링/xgb_prodmodel_cv{cnt}_production.jpg')
    plt.show()
    
    print('-'*100)
    
print(f'교차검증 평균 r2: {np.mean(r2_list)}')
print(f'교차검증 평균 mae: {np.mean(mae_list)}')
print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')

cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
cv_scores['MAE']=mae_list
cv_scores['RMSE']=rmse_list
mean_values = cv_scores.mean()
cv_scores.loc['Average'] = mean_values
cv_scores.to_csv('./최종모델링/xgb_prodmodel_scores.csv')

# DecisionTree

## 교차검증

In [ ]:
## 소비 모델
kfold=KFold(n_splits=7, shuffle=True)
r2_list=[]
mae_list=[]
rmse_list=[]

data_cons=data[data['is_consumption']==1].drop(['data_block_id','row_id'], axis=1)
cnt=0
for tr_idx, ts_idx in kfold.split(data_cons):
    cnt+=1
    x_train, x_test= data_cons.iloc[tr_idx,1:], data_cons.iloc[ts_idx,1:]
    y_train, y_test= data_cons.iloc[tr_idx,0], data_cons.iloc[ts_idx,0]
    
    x_scaler=MinMaxScaler()
    x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
    x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
    y_scaler=MinMaxScaler()
    y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
    
    # print(tr_idx,ts_idx)
    # display( x_test)
    model=DecisionTreeRegressor()
    model.fit(x_train_sc, y_train_sc)
    
    feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_names_in_).sort_values(ascending=False)
    # display(feat_imp)
    # plt.figure(figsize=(24,8))
    sns.barplot(x= feat_imp, y=feat_imp.index)
    plt.title(f'소비모형 교차검증 {cnt}회차 피쳐중요도')
    plt.savefig(f'./최종모델링/dtr_consmodel_cv{cnt}_feature_importances.jpg')
    plt.show()

    y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
    x_test['Actual']=y_test
    x_test['Prediction']=y_pred
    
    r2=r2_score(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))

    r2_list.append(r2)
    mae_list.append(mae)
    rmse_list.append(rmse)


    print(f'{cnt}회차 r2: {r2}')
    print(f'{cnt}회차 mae: {mae}')
    print(f'{cnt}회차 rmse: {rmse}')

    sns.scatterplot(x_test, x='Actual', y='Prediction', color='tomato')
    sns.lineplot(x_test, x='Actual', y='Actual', color='olive')
    plt.title(f'소비모형 교차검증 {cnt}회차 소비그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
    plt.savefig(f'./최종모델링/dtr_consmodel_cv{cnt}_consumption.jpg')
    plt.show()
    
    print('-'*100)
    
print(f'교차검증 평균 r2: {np.mean(r2_list)}')
print(f'교차검증 평균 mae: {np.mean(mae_list)}')
print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')

cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
cv_scores['MAE']=mae_list
cv_scores['RMSE']=rmse_list
mean_values = cv_scores.mean()
cv_scores.loc['Average'] = mean_values
cv_scores.to_csv('./최종모델링/dtr_consmodel_scores.csv')

In [ ]:
## 생산 모델
kfold=KFold(n_splits=7, shuffle=True)
r2_list=[]
mae_list=[]
rmse_list=[]

data_prod=data[data['is_consumption']==0].drop(['data_block_id','row_id'], axis=1)
cnt=0
for tr_idx, ts_idx in kfold.split(data_prod):
    cnt+=1
    x_train, x_test= data_prod.iloc[tr_idx,1:], data_prod.iloc[ts_idx,1:]
    y_train, y_test= data_prod.iloc[tr_idx,0], data_prod.iloc[ts_idx,0]
    
    x_scaler=MinMaxScaler()
    x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
    x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
    y_scaler=MinMaxScaler()
    y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
    
    # print(tr_idx,ts_idx)
    # display( x_test)
    model=DecisionTreeRegressor()
    model.fit(x_train_sc, y_train_sc)
    
    feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_names_in_).sort_values(ascending=False)
    # display(feat_imp)
    # plt.figure(figsize=(24,8))
    sns.barplot(x= feat_imp, y=feat_imp.index)
    plt.title(f'생산모형 교차검증 {cnt}회차 피쳐중요도')
    plt.savefig(f'./최종모델링/dtr_prodmodel_cv{cnt}_feature_importances.jpg')
    plt.show()

    y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
    x_test['Actual']=y_test
    x_test['Prediction']=y_pred
    
    r2=r2_score(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))

    r2_list.append(r2)
    mae_list.append(mae)
    rmse_list.append(rmse)


    print(f'{cnt}회차 r2: {r2}')
    print(f'{cnt}회차 mae: {mae}')
    print(f'{cnt}회차 rmse: {rmse}')

    sns.scatterplot(x_test, x='Actual', y='Prediction', color='deepskyblue')
    sns.lineplot(x_test, x='Actual', y='Actual', color='olive')
    plt.title(f'생산모형 교차검증 {cnt}회차 생산그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
    plt.savefig(f'./최종모델링/dtr_prodmodel_cv{cnt}_production.jpg')
    plt.show()
    
    print('-'*100)
    
print(f'교차검증 평균 r2: {np.mean(r2_list)}')
print(f'교차검증 평균 mae: {np.mean(mae_list)}')
print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')

cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
cv_scores['MAE']=mae_list
cv_scores['RMSE']=rmse_list
mean_values = cv_scores.mean()
cv_scores.loc['Average'] = mean_values
cv_scores.to_csv('./최종모델링/dtr_prodmodel_scores.csv')

# KNN

## 교차검증

In [ ]:
## 소비 모델
kfold=KFold(n_splits=5, shuffle=True)
r2_list=[]
mae_list=[]
rmse_list=[]

data_cons=data[data['is_consumption']==1].drop(['data_block_id','row_id'], axis=1)
cnt=0
for tr_idx, ts_idx in kfold.split(data_cons):
    cnt+=1
    x_train, x_test= data_cons.iloc[tr_idx,1:], data_cons.iloc[ts_idx,1:]
    y_train, y_test= data_cons.iloc[tr_idx,0], data_cons.iloc[ts_idx,0]
    
    x_scaler=MinMaxScaler()
    x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
    x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
    y_scaler=MinMaxScaler()
    y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
    
    # print(tr_idx,ts_idx)
    # display( x_test)
    model=KNeighborsRegressor()
    model.fit(x_train_sc, y_train_sc)
    
    # feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_names_in_).sort_values(ascending=False)
    # # display(feat_imp)
    # # plt.figure(figsize=(24,8))
    # sns.barplot(x= feat_imp, y=feat_imp.index)
    # plt.title(f'소비모형 교차검증 {cnt}회차 피쳐중요도')
    # plt.savefig(f'./최종모델링/knn_consmodel_cv{cnt}_feature_importances.jpg')
    # plt.show()

    y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
    x_test['Actual']=y_test
    x_test['Prediction']=y_pred
    
    r2=r2_score(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))

    r2_list.append(r2)
    mae_list.append(mae)
    rmse_list.append(rmse)


    print(f'{cnt}회차 r2: {r2}')
    print(f'{cnt}회차 mae: {mae}')
    print(f'{cnt}회차 rmse: {rmse}')

    sns.scatterplot(x_test, x='Actual', y='Prediction', color='tomato')
    sns.lineplot(x_test, x='Actual', y='Actual', color='olive')
    plt.title(f'소비모형 교차검증 {cnt}회차 소비그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
    plt.savefig(f'./최종모델링/knn_consmodel_cv{cnt}_consumption.jpg')
    plt.show()
    
    print('-'*100)
    
print(f'교차검증 평균 r2: {np.mean(r2_list)}')
print(f'교차검증 평균 mae: {np.mean(mae_list)}')
print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')

cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
cv_scores['MAE']=mae_list
cv_scores['RMSE']=rmse_list
mean_values = cv_scores.mean()
cv_scores.loc['Average'] = mean_values
cv_scores.to_csv('./최종모델링/knn_consmodel_scores.csv')

In [ ]:
## 생산 모델
kfold=KFold(n_splits=5, shuffle=True)
r2_list=[]
mae_list=[]
rmse_list=[]

data_prod=data[data['is_consumption']==0].drop(['data_block_id','row_id'], axis=1)
cnt=0
for tr_idx, ts_idx in kfold.split(data_prod):
    cnt+=1
    x_train, x_test= data_prod.iloc[tr_idx,1:], data_prod.iloc[ts_idx,1:]
    y_train, y_test= data_prod.iloc[tr_idx,0], data_prod.iloc[ts_idx,0]
    
    x_scaler=MinMaxScaler()
    x_train_sc=pd.DataFrame(x_scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
    x_test_sc=pd.DataFrame(x_scaler.transform(x_test),index=x_test.index, columns=x_test.columns)
    y_scaler=MinMaxScaler()
    y_train_sc=y_scaler.fit_transform(y_train.values.reshape(-1,1))
    
    # print(tr_idx,ts_idx)
    # display( x_test)
    model=KNeighborsRegressor()
    model.fit(x_train_sc, y_train_sc)
    
    # feat_imp=pd.Series(data=model.feature_importances_, index=model.feature_names_in_).sort_values(ascending=False)
    # # display(feat_imp)
    # # plt.figure(figsize=(24,8))
    # sns.barplot(x= feat_imp, y=feat_imp.index)
    # plt.title(f'생산모형 교차검증 {cnt}회차 피쳐중요도')
    # plt.savefig(f'./최종모델링/dtr_prodmodel_cv{cnt}_feature_importances.jpg')
    # plt.show()

    y_pred=y_scaler.inverse_transform(model.predict(x_test_sc.values).reshape(-1,1))
    x_test['Actual']=y_test
    x_test['Prediction']=y_pred
    
    r2=r2_score(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))

    r2_list.append(r2)
    mae_list.append(mae)
    rmse_list.append(rmse)


    print(f'{cnt}회차 r2: {r2}')
    print(f'{cnt}회차 mae: {mae}')
    print(f'{cnt}회차 rmse: {rmse}')

    sns.scatterplot(x_test, x='Actual', y='Prediction', color='deepskyblue')
    sns.lineplot(x_test, x='Actual', y='Actual', color='olive')
    plt.title(f'생산모형 교차검증 {cnt}회차 생산그래프\nr2: {r2:.3f}, mae: {mae:.3f}, rmse: {rmse:.3f}')
    plt.savefig(f'./최종모델링/dtr_prodmodel_cv{cnt}_production.jpg')
    plt.show()
    
    print('-'*100)
    
print(f'교차검증 평균 r2: {np.mean(r2_list)}')
print(f'교차검증 평균 mae: {np.mean(mae_list)}')
print(f'교차검증 평균 rmse: {np.mean(rmse_list)}')

cv_scores=pd.DataFrame(np.round(r2_list,3), columns=['R_square'], index=range(1,8))
cv_scores['MAE']=mae_list
cv_scores['RMSE']=rmse_list
mean_values = cv_scores.mean()
cv_scores.loc['Average'] = mean_values
cv_scores.to_csv('./최종모델링/dtr_prodmodel_scores.csv')

In [ ]:
df_try=df.reset_index().rename(columns={'datetime':'forecast_datetime'})
df_try['forecast_datetime']=pd.to_datetime(df_try['forecast_datetime'])
df_try['origin_datetime']=df_try['forecast_datetime']-pd.DateOffset(hours=48)
df_try

In [ ]:
df_try1=df_try[((df_try['prediction_unit_id']==5)&(df_try['is_consumption']==0)&(df_try['data_block_id']==235))][['forecast_datetime','target','origin_datetime','direct_solar_radiation_f']]

In [ ]:
display(df_try1.head(100))

In [ ]:
sns.histplot(df, x='euros_per_mwh_electricity', bins=50)
plt.savefig('ele_price_histo.jpg')

In [ ]:
sns.histplot(df, x='installed_capacity', bins=50)
plt.savefig('eic_count_histo.jpg')

In [ ]:
sns.lineplot(df.groupby('data_block_id')[['data_block_id','direct_solar_radiation_f']].mean(), x='data_block_id', y='direct_solar_radiation_f', color='orange')
plt.savefig('solar_line.jpg')

In [ ]:
df.info()